In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

import keras
import os
import sys

sys.path.append('..')
import data
import model
import preprocessing
import utils

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.
C:\Users\PC\Anaconda3\envs\TensorFlow-GPU-Keras\lib\site-packages\pydub\utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [6]:
params = utils.yaml_to_dict(os.path.join('..','config.yml'))
params['data_dir'] = os.path.join('..',params['data_dir'])
params['model_dir'] = os.path.join('..',params['model_dir'])

## Primer paso: preprocesar los datos 
Hacerlo solo una vez

In [7]:
#preprocessing.download_data(params)
#preprocessing.extract_data(params)
#preprocessing.generate_spectogram_images(params)
#preprocessing.make_id_label_map(params)
preprocessing.split_data(params)

## Segund Paso: instanciar el modelo

In [8]:
inputs = tf.keras.layers.Input(shape=(110,480, 3))
net = model.ModelArchitecture(num_classes=params['num_classes'])
x = net(inputs, training=False)
net.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 55, 240, 64)       9472      
_________________________________________________________________
batch_normalization_2 (Batch (None, 55, 240, 64)       256       
_________________________________________________________________
activation_3 (Activation)    (None, 55, 240, 64)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 48, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 33792)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               4325504   
_________________________________________________________________
batch_normalization_3 (Batch (None, 128)               512       
__________

## Tercer paso: crear el generador

In [9]:
print('Data in train')
train_generator = data.make_datagenerator(params, mode='training')
print('Data in validation')
val_generator = data.make_datagenerator(params,mode='validation')

Data in train
Found 2960 images.
Data in validation
Found 30 images.


## Cuarto paso: correr el modelo

In [10]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    os.path.join(params['model_dir'], 'tf_ckpt'), 
    save_weights_only=True, 
    verbose=1,
    period=5)

tb_callback = tf.keras.callbacks.TensorBoard(
    os.path.join(params['model_dir'], 'logs'))

optimizer = tf.keras.optimizers.Adam(params['learning_rate'])

steps_per_epoch = train_generator.n // params['batch_size']
validation_steps = val_generator.n // params['batch_size']

net.compile(optimizer=optimizer, loss=params['loss'], metrics=['sparse_categorical_accuracy'])
net.fit_generator(
    train_generator, 
    steps_per_epoch=steps_per_epoch, 
    epochs=params['num_epochs'],
    workers=4,
    validation_data=val_generator, 
    validation_steps=validation_steps,
    callbacks=[cp_callback,tb_callback])

Epoch 1/25
296/296 [==============================] - 35s 117ms/step - loss: 0.5575 - sparse_categorical_accuracy: 0.8162 - val_loss: 0.5467 - val_sparse_categorical_accuracy: 0.9000
Epoch 2/25
296/296 [==============================] - 23s 77ms/step - loss: 0.1298 - sparse_categorical_accuracy: 0.9588 - val_loss: 0.3847 - val_sparse_categorical_accuracy: 0.9000
Epoch 3/25
296/296 [==============================] - 21s 72ms/step - loss: 0.0617 - sparse_categorical_accuracy: 0.9801 - val_loss: 0.3393 - val_sparse_categorical_accuracy: 0.9333
Epoch 4/25
296/296 [==============================] - 19s 64ms/step - loss: 0.0378 - sparse_categorical_accuracy: 0.9885 - val_loss: 0.5760 - val_sparse_categorical_accuracy: 0.9333
Epoch 5/25
295/296 [============================>.] - ETA: 0s - loss: 0.0620 - sparse_categorical_accuracy: 0.9790
Epoch 00005: saving model to ..\checkpoints\tf_ckpt

Consider using a TensorFlow optimizer from `tf.train`.
296/296 [==============================] - 21s 6

296/296 [==============================] - 23s 76ms/step - loss: 4.3017e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.4962 - val_sparse_categorical_accuracy: 0.9333
Epoch 11/25
296/296 [==============================] - 22s 74ms/step - loss: 2.9919e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.4619 - val_sparse_categorical_accuracy: 0.9333
Epoch 12/25
296/296 [==============================] - 21s 71ms/step - loss: 2.5148e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.5127 - val_sparse_categorical_accuracy: 0.9333
Epoch 13/25
296/296 [==============================] - 20s 66ms/step - loss: 2.2128e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.4667 - val_sparse_categorical_accuracy: 0.9333
Epoch 14/25
296/296 [==============================] - 19s 64ms/step - loss: 1.5802e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.5241 - val_sparse_categorical_accuracy: 0.9333
Epoch 15/25
294/296 [============================>.] - ETA: 0s - loss: 1.3109